# Downloading Data with VSO

In this session we will be using the Virtual Solar Observatory (VSO) cilent in SunPy to download some EUV data which we will use in the next session. The VSO has access to a wide variety of different instruments and data sources, and the SunPy client allows you to create arbitarily complex queries to search and then download the data.

## The VSO Module

SunPy's VSO submodule comes in two important parts, the `VSOClient` iteself, the thing that actually does the searching and donwloading of data and the 'attributes' which are things that allow you to build searches, they are things like `Time` or `Instrument`. We import them both: 

In [ ]:
from sunpy.net.vso import VSOClient, attrs as a
import astropy.units as u

Create a VSO client:

In [ ]:
vc = VSOClient()

Let's do a simple search for a days worth of AIA data (don't worry it wont download it all):

In [ ]:
results = vc.query(a.Time("2016/02/03", "2016/02/04"), a.Instrument('AIA'))

In [ ]:
results

This is a little too much data, so let's restrict it to a single wavelength and ask for a lower cadence:

In [ ]:
results = vc.query(a.Time("2016/03/02", "2016/03/03"), a.Instrument('AIA'), a.Wave(17.1*u.nm, 17.1*u.nm),
                   a.Sample(2*u.h))

In [ ]:
results

In [ ]:
files = vc.get(results).wait(progress=True)

## SunPy Mapcube

A quick aside into plotting sequences of images with SunPy. We will load these files we just downloaded into a SunPy MapCube and animate them. (Note is a little more responsive outside of the notebook.)

In [ ]:
%matplotlib notebook
import sunpy.map

In [ ]:
mapcube = sunpy.map.Map(files, cube=True)

In [ ]:
mapcube.peek(resample=(0.25, 0.25))

## More Complex Searching

In this example we will use the logical operators to combine attributes into a complex multi-instrument query.

You can use the `&` and operator or the `|` or operator to make queries. In the previous example all arguments to `vc.query` had the and operator applied, we performed a search where the Instrument *and* the Wavelength *and* the time matched. We could have done this explicitly like this:

In [ ]:
mysearch = a.Time("2016/02/03", "2016/02/03T00:10:00") & a.Instrument('AIA')
vc.query(mysearch)

If we want to do the same query but for two seperate wavelengths we can use the `|` or operator:

In [ ]:
instrument = a.Time("2016/02/03", "2016/02/03T00:10:00") & a.Instrument('AIA')
wavelength = a.Wave(17.1*u.nm, 17.1*u.nm) | a.Wave(304*u.AA, 304*u.AA)
mysearch = instrument & wavelength

In [ ]:
vc.query(mysearch)

### Multiple Instruments

In this example we want to download one image from STEREO A EUVI and one image from SDO AIA as close together in time as we can. To do this we will define a search for the AIA image and a search for the EUVI image then 'or' them together.

In [ ]:
stereo = (a.Source('STEREO_B') &
          a.Instrument('EUVI') &
          a.Time('2011-01-01', '2011-01-01T00:10:00'))

aia = (a.Instrument('AIA') &
       a.Sample(24 * u.hour) &
       a.Time('2011-01-01', '2011-01-02'))

wave = a.Wave(30 * u.nm, 31 * u.nm)

In [ ]:
results = vc.query(stereo | aia, wave)
results

In [ ]:
files = vc.get(result).wait(progress=True)

In [ ]:
files